In [1]:
!pip install pulp
from pulp import *
from scipy.stats import norm

In [2]:
# model
m = LpProblem("EnergySupplier", LpMinimize)

In [3]:
# data
scenarios = {'normal': [5,100],
             'cold': [8,150],
             'very_cold': [10,180]
            }
l = 0.2

In [4]:
# variables 
x_order = LpVariable("order_amount_period1", 0)
x_sold = LpVariable("sold_amount_period1", 0)
y_order = LpVariable.dicts("order_amount_period2",scenarios, 0)

In [5]:
# objective function
m += 1/len(scenarios) * lpSum([y_order[s]*scenarios[s][0] for s in scenarios]
                             ) + x_order * scenarios['normal'][0] +(x_order-x_sold) * l

# constraints
m += x_sold >= scenarios['normal'][1]  ,"demand_period1"
m += x_sold <= x_order, "order_period1"

for s in scenarios:
    m += x_order-x_sold + y_order[s] >= scenarios[s][1] , '{}'.format(s)

In [7]:
# Optimize
m.solve()
print("Status = %s" % LpStatus[m.status])

# Solution
print("Objective = %f" % value(m.objective))
for var in m.variables():
    print("%s = %f" % (var,var.varValue))

Status = Optimal
Objective = 1380.000000
order_amount_period1 = 250.000000
order_amount_period2_cold = 0.000000
order_amount_period2_normal = 0.000000
order_amount_period2_very_cold = 30.000000
sold_amount_period1 = 100.000000
